# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.

# About the Dataset of Google Play Store Apps & Reviews

**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](
https://www.kaggle.com/lava18/google-play-store-apps).

# Import Statements

In [56]:
import pandas as pd
import plotly.express as px

# Notebook Presentation

In [57]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format

# Read the Dataset

In [58]:
df_apps = pd.read_csv('apps.csv')

# Data Cleaning

**Challenge**: How many rows and columns does `df_apps` have? What are the column names? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).

In [59]:
print(f"Shape of dataframe: {df_apps.shape}")

Shape of dataframe: (10841, 12)


In [60]:
df_apps.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size_MBs', 'Installs', 'Type',
       'Price', 'Content_Rating', 'Genres', 'Last_Updated', 'Android_Ver'],
      dtype='object')

In [61]:
df_apps.sample(5)
# Getting 5 random rows

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
6370,10 Best Foods for You,HEALTH_AND_FITNESS,4.00,2490,3.80,"500,000",Free,0,Everyone 10+,Health & Fitness,"February 17, 2017",2.3.3 and up
3173,EZ Pass,TOOLS,2.50,15,0.22,"5,000",Free,0,Everyone,Tools,"September 10, 2016",2.3 and up
7675,The translator,TOOLS,4.50,23805,4.10,"1,000,000",Free,0,Everyone,Tools,"August 13, 2017",4.0.3 and up
9577,Video Downloader,VIDEO_PLAYERS,4.20,59089,5.40,"10,000,000",Free,0,Everyone,Video Players & Editors,"August 3, 2018",4.4 and up
845,P-Home for KLWP,PERSONALIZATION,5.00,4,12.00,100,Paid,$0.99,Everyone,Personalization,"May 15, 2018",5.0 and up


### Drop Unused Columns

**Challenge**: Remove the columns called `Last_Updated` and `Android_Version` from the DataFrame. We will not use these columns. 

In [62]:
df_apps = df_apps.drop(['Last_Updated', 'Android_Ver'], axis=1)

### Find and Remove NaN values in Ratings

**Challenge**: How may rows have a NaN value (not-a-number) in the Ratings column? Create DataFrame called `df_apps_clean` that does not include these rows. 

In [63]:
df_apps.isna().values.sum()
df_apps_clean = df_apps.dropna()

In [64]:
print(f"No. of NaN values in clean df: {df_apps_clean.isna().values.sum()}")
df_apps_clean.shape

No. of NaN values in clean df: 0


(9367, 10)

### Find and Remove Duplicates

**Challenge**: Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`. 


In [65]:
duplicated_rows = df_apps_clean[df_apps_clean.duplicated()]

In [66]:
duplicated_rows.head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
946,420 BZ Budeze Delivery,MEDICAL,5.00,2,11.00,100,Free,0,Mature 17+,Medical
1133,MouseMingle,DATING,2.70,3,3.90,100,Free,0,Mature 17+,Dating
1196,"Cardiac diagnosis (heart rate, arrhythmia)",MEDICAL,4.40,8,6.50,100,Paid,$12.99,Everyone,Medical
1231,Sway Medical,MEDICAL,5.00,3,22.00,100,Free,0,Everyone,Medical
1247,Chat Kids - Chat Room For Kids,DATING,4.70,6,4.90,100,Free,0,Mature 17+,Dating


### So how do we get rid of duplicates? Can we simply call .drop_duplicates()?

`df_apps_clean = df_apps_clean.drop_duplicates()`

Not really, If we do this without specifying how to identify duplicates, we see that 3 copies of Instagram are retained because they have a different number of reviews. We need to provide the column names that should be used in the comparison to identify duplicates. 

In [67]:
df_apps_clean = df_apps_clean.drop_duplicates(subset=['App', 'Type', 'Price'])

In [68]:
# Just checking if duplicates were removed or not
df_apps_clean[df_apps_clean.App == "Instagram"]

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10806,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social


In [69]:
# Checking for the final number of entries to work with
df_apps_clean.shape

(8199, 10)

# Find Highest Rated Apps

**Challenge**: Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?

In [70]:
df_apps_clean.sample()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
3298,FR: My Famous Lover,FAMILY,4.00,185,28.00,"10,000",Free,0,Teen,Entertainment


In [71]:
df_apps_clean.sort_values('Rating', ascending=False)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
21,KBA-EZ Health Guide,MEDICAL,5.00,4,25.00,1,Free,0,Everyone,Medical
1751,Spine- The dating app,DATING,5.00,5,9.30,500,Free,0,Teen,Dating
1761,Selfie With Champion AJ Style,PHOTOGRAPHY,5.00,2,7.50,500,Free,0,Everyone,Photography
1763,Ag Valley Cooperative,BUSINESS,5.00,6,74.00,500,Free,0,Everyone,Business
1769,Yazdani Cd Center EllahAbad Official App,FAMILY,5.00,8,3.80,500,Free,0,Everyone,Entertainment
...,...,...,...,...,...,...,...,...,...,...
818,Familial Hypercholesterolaemia Handbook,MEDICAL,1.00,2,33.00,100,Free,0,Everyone,Medical
1208,Thistletown CI,PRODUCTIVITY,1.00,1,6.60,100,Free,0,Everyone,Productivity
1314,CR Magazine,BUSINESS,1.00,1,7.80,100,Free,0,Everyone,Business
1679,CB Mobile Biz,FINANCE,1.00,3,8.40,500,Free,0,Everyone,Finance


**No we can't totally rely on reviews**, because it does not take into account the number of reviews given.

# Find 5 Largest Apps in terms of Size (MBs)

**Challenge**: What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be limit in place or can developers make apps as large as they please? 

In [72]:
df_apps_clean.sort_values('Size_MBs', ascending=False)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
8719,Draft Simulator for FUT 18,SPORTS,4.60,162933,100.00,"5,000,000",Free,0,Everyone,Sports
8718,Mini Golf King - Multiplayer Game,GAME,4.50,531458,100.00,"5,000,000",Free,0,Everyone,Sports
10295,SimCity BuildIt,FAMILY,4.50,4218587,100.00,"50,000,000",Free,0,Everyone 10+,Simulation
9945,Ultimate Tennis,SPORTS,4.30,183004,100.00,"10,000,000",Free,0,Everyone,Sports
9944,Gangster Town: Vice District,FAMILY,4.30,65146,100.00,"10,000,000",Free,0,Mature 17+,Simulation
...,...,...,...,...,...,...,...,...,...,...
5798,ExDialer PRO Key,COMMUNICATION,4.50,5474,0.02,"100,000",Paid,$3.99,Everyone,Communication
2648,Ad Remove Plugin for App2SD,PRODUCTIVITY,4.10,66,0.02,"1,000",Paid,$1.29,Everyone,Productivity
2684,My baby firework (Remove ad),FAMILY,4.10,30,0.01,"1,000",Paid,$0.99,Everyone,Entertainment
7966,Market Update Helper,LIBRARIES_AND_DEMO,4.10,20145,0.01,"1,000,000",Free,0,Everyone,Libraries & Demo


According to the database provided it feels like that the **limit for size of an app is set to be 100MB**

# Find the 5 App with Most Reviews

**Challenge**: Which apps have the highest number of reviews? Are there any paid apps among the top 50?

In [73]:
top_50_most_reviewed = df_apps_clean.sort_values('Reviews', ascending=False).head(50)
print(f"Number of Paid Apps in top 50 reviewed apps: {top_50_most_reviewed[top_50_most_reviewed.Type == 'Paid'].App.count()}")
top_50_most_reviewed.head()

Number of Paid Apps in top 50 reviewed apps: 0


,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10805,Facebook,SOCIAL,4.10,78158306,5.30,"1,000,000,000",Free,0,Teen,Social
10785,WhatsApp Messenger,COMMUNICATION,4.40,69119316,3.50,"1,000,000,000",Free,0,Everyone,Communication
10806,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10784,Messenger – Text and Video Chat for Free,COMMUNICATION,4.00,56642847,3.50,"1,000,000,000",Free,0,Everyone,Communication
10650,Clash of Clans,GAME,4.60,44891723,98.00,"100,000,000",Free,0,Everyone 10+,Strategy


# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings

In [74]:
# Lets see all kind of different apps 
rating = df_apps_clean.Content_Rating.value_counts()
rating

Content_Rating
Everyone           6621
Teen                912
Mature 17+          357
Everyone 10+        305
Adults only 18+       3
Unrated               1
Name: count, dtype: int64

In [75]:
# To create a pie chart we call px.pie()
fig = px.pie(
  labels=rating.index, 
  values=rating.values, 
  title='Content Rating',
  names=rating.index)

# To show names with the percentage
fig.update_traces(textposition='outside', textinfo='percent+label')

fig.show()

In [76]:
# To create a donut chart we call px.pie() and put a hole in it
fig = px.pie(
  labels=rating.index, 
  values=rating.values, 
  title='Content Rating',
  names=rating.index,
  hole=0.5)

# To show names with the percentage
fig.update_traces(textposition='inside', textinfo='percent', textfont_size=12)

fig.show()

# Numeric Type Conversion: Examine the Number of Installs

**Challenge**: How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install? 

Check the datatype of the Installs column.

Count the number of apps at each level of installations. 

Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have make sure you remove non-numeric characters first. 

In [77]:
df_apps_clean.sample()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
1805,FindLoving,DATING,3.80,24,19.00,"1,000",Free,0,Mature 17+,Dating


In [78]:
# To get info about the datatypes of all the columns 
df_apps_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8199 entries, 21 to 10835
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             8199 non-null   object 
 1   Category        8199 non-null   object 
 2   Rating          8199 non-null   float64
 3   Reviews         8199 non-null   int64  
 4   Size_MBs        8199 non-null   float64
 5   Installs        8199 non-null   object 
 6   Type            8199 non-null   object 
 7   Price           8199 non-null   object 
 8   Content_Rating  8199 non-null   object 
 9   Genres          8199 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 704.6+ KB


In [79]:
df_apps_clean.Installs = df_apps_clean.Installs.astype(str).str.replace(',', '')
df_apps_clean.Installs = pd.to_numeric(df_apps_clean.Installs)
df_apps_clean.Installs.tail()

10824    1000000000
10828    1000000000
10829    1000000000
10831    1000000000
10835    1000000000
Name: Installs, dtype: int64

In [80]:
# Number of apps with more than 1B+ downloads
print(f"No. of apps with 1B downloads: {df_apps_clean[df_apps_clean.Installs == 1000000000].Installs.count()}")

No. of apps with 1B downloads: 20


# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

Let's examine the Price column more closely.

**Challenge**: Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.

Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.

Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest grossing paid apps according to this estimate? Out of the top 10 highest grossing paid apps, how many are games?


In [81]:
df_apps_clean.Price = df_apps_clean.Price.astype(str).str.replace('$', '')
df_apps_clean.Price = pd.to_numeric(df_apps_clean.Price)
df_apps_clean.insert(8, 'Revenue_Estimate', (df_apps_clean.Price)*(df_apps_clean.Installs))

In [82]:
df_apps_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8199 entries, 21 to 10835
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   App               8199 non-null   object 
 1   Category          8199 non-null   object 
 2   Rating            8199 non-null   float64
 3   Reviews           8199 non-null   int64  
 4   Size_MBs          8199 non-null   float64
 5   Installs          8199 non-null   int64  
 6   Type              8199 non-null   object 
 7   Price             8199 non-null   float64
 8   Revenue_Estimate  8199 non-null   float64
 9   Content_Rating    8199 non-null   object 
 10  Genres            8199 non-null   object 
dtypes: float64(4), int64(2), object(5)
memory usage: 768.7+ KB


### The most expensive apps sub $250

In [83]:
df_apps_clean[df_apps_clean.Price >= 250].sort_values('Price', ascending=False)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Revenue_Estimate,Content_Rating,Genres
3946,I'm Rich - Trump Edition,LIFESTYLE,3.60,275,7.30,10000,Paid,400.00,"4,000,000.00",Everyone,Lifestyle
1946,I am rich (Most expensive app),FINANCE,4.10,129,2.70,1000,Paid,399.99,"399,990.00",Teen,Finance
1331,most expensive app (H),FAMILY,4.30,6,1.50,100,Paid,399.99,"39,999.00",Everyone,Entertainment
2775,I Am Rich Pro,FAMILY,4.40,201,2.70,5000,Paid,399.99,"1,999,950.00",Everyone,Entertainment
2461,I AM RICH PRO PLUS,FINANCE,4.00,36,41.00,1000,Paid,399.99,"399,990.00",Everyone,Finance
3114,I am Rich,FINANCE,4.30,180,3.80,5000,Paid,399.99,"1,999,950.00",Everyone,Finance
2394,I am Rich!,FINANCE,3.80,93,22.00,1000,Paid,399.99,"399,990.00",Everyone,Finance
5765,I am rich,LIFESTYLE,3.80,3547,1.80,100000,Paid,399.99,"39,999,000.00",Everyone,Lifestyle
3145,I am rich(premium),FINANCE,3.50,472,0.94,5000,Paid,399.99,"1,999,950.00",Everyone,Finance
3554,💎 I'm rich,LIFESTYLE,3.80,718,26.00,10000,Paid,399.99,"3,999,900.00",Everyone,Lifestyle


In [84]:
# These apps seems wrong or say manipulated so we are going to remove these apps
df_apps_clean = df_apps_clean[df_apps_clean.Price < 250]

### Highest Grossing Paid Apps (ballpark estimate)

In [85]:
df_apps_clean.sort_values('Revenue_Estimate', ascending=False)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Revenue_Estimate,Content_Rating,Genres
9220,Minecraft,FAMILY,4.50,2376564,19.00,10000000,Paid,6.99,"69,900,000.00",Everyone 10+,Arcade;Action & Adventure
8825,Hitman Sniper,GAME,4.60,408292,29.00,10000000,Paid,0.99,"9,900,000.00",Mature 17+,Action
7151,Grand Theft Auto: San Andreas,GAME,4.40,348962,26.00,1000000,Paid,6.99,"6,990,000.00",Mature 17+,Action
7477,Facetune - For Free,PHOTOGRAPHY,4.40,49553,48.00,1000000,Paid,5.99,"5,990,000.00",Everyone,Photography
7977,Sleep as Android Unlock,LIFESTYLE,4.50,23966,0.85,1000000,Paid,5.99,"5,990,000.00",Everyone,Lifestyle
...,...,...,...,...,...,...,...,...,...,...,...
4632,Inputting Plus: Ctrl + Z/F/C/V,PRODUCTIVITY,4.20,1117,3.00,50000,Free,0.00,0.00,Everyone,Productivity
4633,Drink-O-Tron The Drinking Game,GAME,4.10,140,45.00,50000,Free,0.00,0.00,Mature 17+,Card
4635,EZ Quran,BOOKS_AND_REFERENCE,4.60,595,8.50,50000,Free,0.00,0.00,Everyone,Books & Reference
4636,Berner Zeitung,NEWS_AND_MAGAZINES,3.40,736,8.50,50000,Free,0.00,0.00,Teen,News & Magazines


# Plotly Bar Charts & Scatter Plots: Analysing App Categories

### Vertical Bar Chart - Highest Competition (Number of Apps)

In [86]:
print(f"Number of unique categories: {df_apps_clean.Category.nunique()}")
print("\nTop 10 Categories are: ")
top_10_categories = df_apps_clean.Category.value_counts()[:10]
top_10_categories

Number of unique categories: 33

Top 10 Categories are: 


Category
FAMILY             1606
GAME                910
TOOLS               719
PRODUCTIVITY        301
PERSONALIZATION     298
LIFESTYLE           297
FINANCE             296
MEDICAL             292
PHOTOGRAPHY         263
BUSINESS            262
Name: count, dtype: int64

In [87]:
# Plotting a bar graph of top 10 categories
bar = px.bar(x=top_10_categories.index, y=top_10_categories.values, title="Top 10 Categories")
bar.update_layout(xaxis_title='Categories', yaxis_title='No of apps')

bar.show()

In [88]:
# What matters is not just the total number of apps in the category but how often apps are downloaded in that category. 
# This will give us an idea of how popular a category is.

category_installs = df_apps_clean.groupby('Category').agg({'Installs': pd.Series.sum})
category_installs = category_installs.sort_values('Installs', ascending=False)
category_installs

,Installs
Category,
GAME,13858762717
COMMUNICATION,11039241530
TOOLS,8099724500
PRODUCTIVITY,5788070180
SOCIAL,5487841475
PHOTOGRAPHY,4649143130
FAMILY,4437554490
VIDEO_PLAYERS,3916897200
TRAVEL_AND_LOCAL,2894859300


### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)

In [89]:
h_bar = px.bar(x = category_installs.Installs,
                y = category_installs.index,
                orientation='h', 
                title='Category Popularity')
h_bar.update_layout(xaxis_title='Number of Downloads', yaxis_title='Category')
  
h_bar.show()

### Category Concentration - Downloads vs. Competition

**Challenge**: 
* First, create a DataFrame that has the number of apps in one column and the number of installs in another:

<img src=https://imgur.com/uQRSlXi.png width="350">

* Then use the [plotly express examples from the documentation](https://plotly.com/python/line-and-scatter/) alongside the [.scatter() API reference](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html)to create scatter plot that looks like this. 

<img src=https://imgur.com/cHsqh6a.png>

*Hint*: Use the size, hover_name and color parameters in .scatter(). To scale the yaxis, call .update_layout() and specify that the yaxis should be on a log-scale like so: yaxis=dict(type='log') 

In [90]:
category_app_installs = df_apps_clean.groupby('Category').agg({'Installs': pd.Series.sum, 'App': pd.Series.count})
category_app_installs = category_app_installs.sort_values('Installs', ascending=False)
category_app_installs

# Plotting Scatter plot
# Setting color as installs to show different color according to number of installs
# Setting size as app so as to see all the different number of apps
scatter = px.scatter(category_app_installs, x='App', y='Installs', color='Installs', size='App', hover_name=category_app_installs.index)
scatter.update_layout(xaxis_title="Number of Apps (Lower=More Concentrated)",
                      yaxis_title="Installs",
                      yaxis=dict(type='log'))

scatter.show()

# Extracting Nested Data from a Column

**Challenge**: How many different types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's [.stack() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html). 


In [91]:
# Getting number of different genres
df_apps_clean.Genres.nunique()
# This is before seperating the nested one

114

In [92]:
df_apps_clean.Genres.value_counts()

Genres
Tools                       718
Entertainment               467
Education                   429
Productivity                301
Personalization             298
                           ... 
Casual;Music & Video          1
Tools;Education               1
Role Playing;Brain Games      1
Adventure;Education           1
Lifestyle;Pretend Play        1
Name: count, Length: 114, dtype: int64

In [121]:
# Split the strings on the semi-colon and then .stack them.
stack = df_apps_clean.Genres.str.split(';', expand=True).stack()
# The expand=True parameter tells Pandas to expand the split strings into separate columns in a new DataFrame.
print(f'We now have a single column with shape: {stack.shape}')
num_genres = stack.value_counts()
print(f'Number of genres: {len(num_genres)}')
num_genres.head(3)

We now have a single column with shape: (8564,)
Number of genres: 53


Tools            719
Education        587
Entertainment    498
Name: count, dtype: int64

**Explaining the output**\
In the genre column data was like: Medical;Arcade, Arcade and Arcade;Medical\
then after stacking it became like we see in the above case as output

**Here it means:**\
`21 0` Medical means that the original row at **index 21** had **"Medical"** as its first genre.\
`28 0` Arcade means that the original row at **index 28** had **"Arcade"** as its first genre.\

# Colour Scales in Plotly Charts - Competition in Genres

**Challenge**: Can you create this chart with the Series containing the genre data? 

<img src=https://imgur.com/DbcoQli.png width=400>

Try experimenting with the built in colour scales in Plotly. You can find a full list [here](https://plotly.com/python/builtin-colorscales/). 

* Find a way to set the colour scale using the color_continuous_scale parameter. 
* Find a way to make the color axis disappear by using coloraxis_showscale. 

In [150]:
bar = px.bar(x=num_genres.index[:15], 
            y=num_genres.values[:15], 
            title='Top Genres', 
            hover_name=num_genres.values[:15],
            color=num_genres.values[:15],  
            color_continuous_scale="Viridis")
bar.update_layout(xaxis_title = 'Genres', yaxis_title = 'No. of Apps', coloraxis_showscale=False)

# Now one issue you might is listed below


bar.show()

# Grouped Bar Charts: Free vs. Paid Apps per Category

In [151]:
df_apps_clean.sample()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Revenue_Estimate,Content_Rating,Genres
6942,Big Days - Events Countdown,LIFESTYLE,4.60,39724,8.35,1000000,Free,0.00,0.00,Everyone,Lifestyle


In [153]:
df_apps_clean.Type.value_counts()

Type
Free    7595
Paid     589
Name: count, dtype: int64

### What we are doing?

**We are basically telling for different combination for (Category, Type) count the number of Apps**

In [156]:
# Categories with paid or free
free_paid_df = df_apps_clean.groupby(['Category', 'Type'], as_index=False).agg({'App': pd.Series.count})
# as_index=False ensures that the group keys ("Category" and "Type") remain as columns instead of becoming index
# Using the above command we are basically telling for different combination for (Category, Type) count the number of Apps
free_paid_df.sort_values('App', ascending=True)

,Category,Type,App
3,AUTO_AND_VEHICLES,Paid,1
17,ENTERTAINMENT,Paid,2
24,FOOD_AND_DRINK,Paid,2
38,NEWS_AND_MAGAZINES,Paid,2
48,SHOPPING,Paid,2
...,...,...,...
31,LIFESTYLE,Free,284
21,FINANCE,Free,289
53,TOOLS,Free,656
25,GAME,Free,834


This basically means that no. of AUTO_AND_VEHICLES Paid App is = 1\
similarly number of free apps which belong to Lifestyle = 284 

**Challenge**: Use the plotly express bar [chart examples](https://plotly.com/python/bar-charts/#bar-chart-with-sorted-or-ordered-categories) and the [.bar() API reference](https://plotly.com/python-api-reference/generated/plotly.express.bar.html#plotly.express.bar) to create this bar chart: 

<img src=https://imgur.com/LE0XCxA.png>

You'll want to use the `df_free_vs_paid` DataFrame that you created above that has the total number of free and paid apps per category. 

See if you can figure out how to get the look above by changing the `categoryorder` to 'total descending' as outlined in the documentation here [here](https://plotly.com/python/categorical-axes/#automatically-sorting-categories-by-name-or-total-value). 

In [183]:
g_bar = px.bar(free_paid_df,
                x='Category',
                y='App',
                title='Free vs Paid Apps by Category',
                color='Type', # Just by adding this line Free apps are gone to seperate bars instead of bars at the top
                barmode='group')
  
g_bar.update_layout(xaxis_title='Category',
                    yaxis_title='Number of Apps',
                    xaxis={'categoryorder':'total descending'},
                    yaxis=dict(type='log')) # Changing y axis to log scale to make the bar graph look good

g_bar.show()

# Plotly Box Plots: Lost Downloads for Paid Apps

**Challenge**: Create a box plot that shows the number of Installs for free versus paid apps. How does the median number of installations compare? Is the difference large or small?

Use the [Box Plots Guide](https://plotly.com/python/box-plots/) and the [.box API reference](https://plotly.com/python-api-reference/generated/plotly.express.box.html) to create the following chart. 

<img src=https://imgur.com/uVsECT3.png>


In [198]:
box = px.box(y=df_apps_clean.Installs, x=df_apps_clean.Type, color=df_apps_clean.Type, 
            title='How many downloads are we giving up?',
            points='all',
            notched=True)
box.update_layout(yaxis=dict(type='log'),
                  yaxis_title='Installs',
                  xaxis_title='Type of App',
                  height=600)

box.show()

# Plotly Box Plots: Revenue by App Category

**Challenge**: See if you can generate the chart below: 

<img src=https://imgur.com/v4CiNqX.png>

Looking at the hover text, how much does the median app earn in the Tools category? If developing an Android app costs $30,000 or thereabouts, does the average photography app recoup its development costs?

Hint: I've used 'min ascending' to sort the categories. 

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

**Challenge**: What is the median price price for a paid app? Then compare pricing by category by creating another box plot. But this time examine the prices (instead of the revenue estimates) of the paid apps. I recommend using `{categoryorder':'max descending'}` to sort the categories.